
# 🪟 Guía rápida de **Window Functions** en SQL

Este **Jupyter Notebook** está pensado para estudiar de forma clara y práctica las *window functions*.
Incluye explicaciones en **Markdown** y ejemplos en **SQL** (puedes copiar y pegar en tu motor SQL).

> **Compatibilidad (general):** PostgreSQL, SQL Server, Oracle, MySQL 8+, SQLite 3.25+. Algunos detalles de sintaxis pueden variar levemente entre motores.

---



## 1) ¿Qué es una *window function*?

Una *window function* **calcula algo mirando un conjunto de filas (la *ventana*)** y **escribe el resultado al lado de cada fila sin agrupar ni eliminar filas**. 

Se escriben con la sintaxis:

```sql
<función_de_ventana>() OVER (
  PARTITION BY ...   -- define "subgrupos lógicos" (opcional)
  ORDER BY ...       -- define el orden en la ventana (opcional)
  [frame clause]     -- define el marco de filas que se miran (opcional)
)
```

- `PARTITION BY` crea **grupos lógicos** sin reducir el número de filas.
- `ORDER BY` define el **orden** dentro de cada partición (clave para ranking, LAG/LEAD, acumulados, etc.).
- El **marco** (`ROWS` o `RANGE`) afina **exactamente qué filas** se consideran alrededor de la fila actual.



## 2) Tabla de ejemplo

Usaremos una tabla **ventas** con las columnas clave:

```sql
-- Esquema sugerido (adáptalo a tu motor)
CREATE TABLE ventas (
  vendedor   VARCHAR(50),
  fecha      DATE,
  monto      DECIMAL(10,2),
  region     VARCHAR(50)
);

-- Datos de ejemplo
INSERT INTO ventas (vendedor, fecha, monto, region) VALUES
('Ana',  '2024-01-05', 100, 'Norte'),
('Ana',  '2024-01-10', 200, 'Norte'),
('Ana',  '2024-02-02', 150, 'Sur'),
('Luis', '2024-01-07', 300, 'Norte'),
('Luis', '2024-01-20', 120, 'Sur'),
('Luis', '2024-02-11', 180, 'Sur'),
('Mia',  '2024-01-15', 130, 'Norte'),
('Mia',  '2024-02-01', 220, 'Norte');
```



## 3) `SUM()` con `OVER()` — Total por vendedor **sin perder filas**

Calcula el total de ventas por vendedor y lo repite en cada fila del vendedor.


In [ ]:

-- Total por vendedor (sin agrupar filas)
SELECT
  vendedor,
  fecha,
  monto,
  SUM(monto) OVER (PARTITION BY vendedor) AS total_por_vendedor
FROM ventas
ORDER BY vendedor, fecha;



## 4) Acumulado (running total) por vendedor

Usa `ORDER BY` para calcular el acumulado en el tiempo dentro de cada vendedor.


In [ ]:

-- Acumulado por vendedor ordenado por fecha
SELECT
  vendedor,
  fecha,
  monto,
  SUM(monto) OVER (
    PARTITION BY vendedor
    ORDER BY fecha
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS acumulado_vendedor
FROM ventas
ORDER BY vendedor, fecha;



## 5) Promedio móvil (ventana deslizante)

Promedio de los **últimos 3 registros** (incluyendo el actual) por vendedor.


In [ ]:

-- Promedio móvil de 3 filas por vendedor
SELECT
  vendedor,
  fecha,
  monto,
  AVG(monto) OVER (
    PARTITION BY vendedor
    ORDER BY fecha
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) AS promedio_movil_3
FROM ventas
ORDER BY vendedor, fecha;



## 6) % del total por región

Divide el monto de cada fila entre el total de su región.


In [ ]:

-- Porcentaje del total regional
SELECT
  region,
  vendedor,
  fecha,
  monto,
  monto * 1.0 / SUM(monto) OVER (PARTITION BY region) AS pct_region
FROM ventas
ORDER BY region, vendedor, fecha;



## 7) `ROW_NUMBER()`, `RANK()`, `DENSE_RANK()`

- `ROW_NUMBER()` numera sin empates (1, 2, 3, ...)
- `RANK()` deja huecos cuando hay empates (1, 1, 3, ...)
- `DENSE_RANK()` no deja huecos (1, 1, 2, ...)

Ejemplo: ranking por monto dentro de cada región (mayor monto primero).


In [ ]:

-- Ranking por región
SELECT
  region,
  vendedor,
  fecha,
  monto,
  ROW_NUMBER()  OVER (PARTITION BY region ORDER BY monto DESC) AS rn,
  RANK()        OVER (PARTITION BY region ORDER BY monto DESC) AS rnk,
  DENSE_RANK()  OVER (PARTITION BY region ORDER BY monto DESC) AS drnk
FROM ventas
ORDER BY region, rnk, vendedor;



## 8) `LAG()` y `LEAD()` — valores anterior y siguiente

Comparar la venta actual con la anterior por vendedor.


In [ ]:

-- Diferencia vs la venta anterior por vendedor
SELECT
  vendedor,
  fecha,
  monto,
  LAG(monto) OVER (PARTITION BY vendedor ORDER BY fecha) AS monto_anterior,
  monto - LAG(monto) OVER (PARTITION BY vendedor ORDER BY fecha) AS diferencia_vs_anterior
FROM ventas
ORDER BY vendedor, fecha;



## 9) `NTILE(n)` — dividir en cuantiles

Divide las filas ordenadas en **n** grupos del mismo tamaño (o casi).


In [ ]:

-- Cuartiles de monto a nivel global
SELECT
  vendedor,
  fecha,
  monto,
  NTILE(4) OVER (ORDER BY monto) AS cuartil
FROM ventas
ORDER BY monto;



## 10) `ROWS` vs `RANGE` en el marco de ventana

- `ROWS` cuenta **filas físicas**.
- `RANGE` agrupa **valores iguales** en la misma marca cuando hay orden por una sola columna numérica.

> Dependiendo del motor, `RANGE` puede requerir `ORDER BY` en una sola expresión y tener diferencias sutiles.


In [ ]:

-- Ejemplo conceptual (tu motor puede variar)
-- Acumulado por fecha global, contando filas físicas
SELECT
  fecha,
  monto,
  SUM(monto) OVER (
    ORDER BY fecha
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS acum_rows
FROM ventas
ORDER BY fecha;

-- Similar con RANGE: agrupa valores con mismo ORDER BY (si aplica y si tu motor lo soporta como esperas)
SELECT
  fecha,
  monto,
  SUM(monto) OVER (
    ORDER BY fecha
    RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS acum_range
FROM ventas
ORDER BY fecha;



## 11) Top-N por grupo con `ROW_NUMBER()`

Quedarse con la **venta más alta por vendedor**.


In [ ]:

-- Top 1 por vendedor (mayor monto)
WITH ranked AS (
  SELECT
    vendedor,
    fecha,
    monto,
    ROW_NUMBER() OVER (PARTITION BY vendedor ORDER BY monto DESC) AS rn
  FROM ventas
)
SELECT *
FROM ranked
WHERE rn = 1
ORDER BY vendedor;



## 12) Particiones múltiples (vendedor + región)

Acumulado por **vendedor y región**.


In [ ]:

-- Acumulado por vendedor y región
SELECT
  vendedor,
  region,
  fecha,
  monto,
  SUM(monto) OVER (
    PARTITION BY vendedor, region
    ORDER BY fecha
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS acumulado_vend_region
FROM ventas
ORDER BY vendedor, region, fecha;



## 13) Nota: `DISTINCT` y *window functions*

`DISTINCT` **no** se aplica dentro de las funciones de ventana como `SUM() OVER (...)`. 
Si necesitas una agregación con `DISTINCT`, normalmente debes hacerlo en una subconsulta o CTE y luego aplicar la *window function* sobre ese resultado.



## 14) Consejos prácticos y diferencias entre motores

- **PostgreSQL / SQL Server / Oracle**: soporte muy completo de *window functions* y marcos (`ROWS`/`RANGE`).
- **MySQL**: desde **8.0** soporta *window functions*. Versiones anteriores **no**.
- **SQLite**: desde **3.25** (2018) tiene *window functions* básicas; revisa limitaciones de `RANGE`.
- **Performance**: agrega índices que soporten el `PARTITION BY`/`ORDER BY` cuando sea posible.
- **`ORDER BY` es clave** para resultados deterministas, sobre todo con `ROW_NUMBER`, `LAG/LEAD` y acumulados.



## 15) Retos para practicar

1. Calcula el **% de contribución** de cada vendedor dentro de su **región y mes**.
2. Obtén el **top 2** de ventas por **región** usando `ROW_NUMBER()`.
3. Calcula la **variación porcentual** vs la venta anterior por vendedor: `(monto - LAG(monto)) / LAG(monto)`.
4. Saca una **media móvil de 7 días** por vendedor.
5. Usa `NTILE(3)` para clasificar a cada venta en **bajo/medio/alto** dentro de cada región.



---

### ✔️ Resumen rápido
- *Window functions* **no reducen filas**, agregan información fila a fila.
- `PARTITION BY` define **subgrupos**, `ORDER BY` define **orden**, y el **marco** (`ROWS`/`RANGE`) define **exactamente qué filas** se consideran.
- Herramientas clave: `SUM/AVG/COUNT` con `OVER`, `ROW_NUMBER`, `RANK/DENSE_RANK`, `LAG/LEAD`, `NTILE`, acumulados y promedios móviles.

¿Quieres que adapte los ejemplos a tu motor (PostgreSQL, SQL Server, MySQL, etc.) o a tu propio esquema de datos?
